<div style="text-align: center;">
    <a href="https://www.dataia.eu/">
        <img border="0" src="https://github.com/ramp-kits/template-kit/raw/main/img/DATAIA-h.png" width="90%"></a>
</div>

# Template Kit for RAMP challenge

<i> Thomas Moreau (Inria) </i>

## Introduction

Describe the challenge, in particular:

- Where the data comes from?
- What is the task this challenge aims to solve?
- Why does it matter?

# Exploratory data analysis

The goal of this section is to show what's in the data, and how to play with it.
This is the first set in any data science project, and here, you should give a sense of the data the participants will be working with.

You can first load and describe the data, and then show some interesting properties of it.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

# Load the data

import problem
X_df, y = problem.get_train_data()

In [2]:
X_df

,GEO,GEO_OBJECT,FREQ,SEX,AGE,EDUC,RP_MEASURE,TIME_PERIOD,EMPSTA_ENQ_1,EMPSTA_ENQ_31,EMPSTA_ENQ_33,EMPSTA_ENQ_35,EMPSTA_ENQ_36
145,02,DEP,A,F,Y15T24,_T,POP,2010,8287.08108,NaN,15176.23846,1311.78823,1237.83905
9,03,DEP,A,M,Y15T24,_T,POP,2010,4734.11560,NaN,8537.65439,10.87200,617.81037
375,01,DEP,A,F,Y15T24,_T,POP,2015,10350.08248,NaN,17272.39775,545.87972,1410.66377
523,53,DEP,A,F,Y15T24,_T,POP,2021,8058.07757,NaN,8311.43191,8.24902,643.74778
188,01,DEP,A,F,Y15T24,_T,POP,2010,11169.40604,NaN,16464.02544,19.54658,862.40590
...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,60,DEP,A,M,Y15T24,_T,POP,2010,20304.00203,1.04050,24328.07940,38.04643,2202.57898
106,86,DEP,A,M,Y15T24,_T,POP,2010,8497.42288,NaN,14439.59619,12.59901,656.11682
270,971,DEP,A,F,Y15T24,_T,POP,2015,3173.34684,NaN,15036.94265,20.48726,1463.29388
435,91,DEP,A,M,Y15T24,_T,POP,2021,27993.71190,NaN,45522.01930,68.13880,5753.73790


# Challenge evaluation

A particularly important point in a challenge is to describe how it is evaluated. This is the section where you should describe the metric that will be used to evaluate the participants' submissions, as well as your evaluation strategy, in particular if there is some complexity in the way the data should be split to ensure valid results.

# Submission format

Here, you should describe the submission format. This is the format the participants should follow to submit their predictions on the RAMP plateform.

This section also show how to use the `ramp-workflow` library to test the submission locally.

## The pipeline workflow

The input data are stored in a dataframe. To go from a dataframe to a numpy array we will use a scikit-learn column transformer. The first example we will write will just consist in selecting a subset of columns we want to work with.

In [5]:
# %load submissions/starting_kit/estimator.py

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer

cols = ['GEO', 'SEX', 'TIME_PERIOD', 'EMPSTA_ENQ_1', 'EMPSTA_ENQ_31', 'EMPSTA_ENQ_33',
       'EMPSTA_ENQ_35', 'EMPSTA_ENQ_36']

categorical_cols = ['GEO', 'SEX', 'TIME_PERIOD']
numerical_cols = ['EMPSTA_ENQ_1', 'EMPSTA_ENQ_31', 'EMPSTA_ENQ_33', 'EMPSTA_ENQ_35', 'EMPSTA_ENQ_36']

transformer = make_column_transformer(
    (OneHotEncoder(handle_unknown='ignore'), categorical_cols),
    ('passthrough', numerical_cols)
)

def get_estimator():
    pipe = make_pipeline(
        transformer,
        SimpleImputer(strategy='most_frequent'),
        LinearRegression()
    )

    return pipe


## Testing using a scikit-learn pipeline

In [8]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(get_estimator(), X_df, y, cv=3, scoring='neg_median_absolute_error')
print(-scores)

[496.0516844  512.16046921 498.74991117]


## Submission

To submit your code, you can refer to the [online documentation](https://paris-saclay-cds.github.io/ramp-docs/ramp-workflow/stable/using_kits.html).